# Simple PyTorch example
---

<font color='red'> <h3>Tested with pytorch 0.3.0</h3></font>
<font color='red'> <h3>Tested with torchvision 0.2.0</h3></font>

In [1]:
def pytorch_mnist(epochs, lr, momentum):
    import argparse
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    from torchvision import datasets, transforms
    from torch.autograd import Variable
    
    import os
    from hops import hdfs
    from hops import devices
    
    batch_size = 64    
    test_batch_size = 1000    
    epochs = 10
    lr = 0.01    
    momentum = 0.5    
    cuda = devices.get_num_gpus() > 0
    seed = 1
    log_interval = 10

    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)
    
    
    # Copy dataset from project to current working directory
    if not os.path.exists(os.getcwd() + '/processed'):
        hdfs.log('----Downloading dataset to Spark executor working directory----')
        # copy_from_project OPTIONAL FLAGS:
        # overwrite: (default false) overwrite the destination path
        # project: (default current project) copy dataset from a different project
        hdfs.copy_from_project('TestJob/data/mnist/processed', '')
        hdfs.log('----------------------Download finished------------------------')

    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.getcwd(), train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.getcwd(), train=False, download=False, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
            self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(320, 50)
            self.fc2 = nn.Linear(50, 10)

        def forward(self, x):
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            return F.log_softmax(x)

    model = Net()
    if cuda:
        model.cuda()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    def train(epoch):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))

    def test():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        accuracy = correct / len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {:.4f} ({:.0f}%)\n'.format(
            test_loss, accuracy,
            100. * correct / len(test_loader.dataset)))
        
        return accuracy

    accuracy = 0
    for epoch in range(1, epochs + 1):
        train(epoch)
        accuracy = test()        
        
    hdfs.log('Epoch: ' + str(epoch) + ' ----- Accuracy: ' + str(accuracy))
    
    return accuracy

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1522241157877_0014,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
# Define hyperparameter search space
args = {'epochs': [1, 20], 'lr': [0.005, 0.05], 'momentum': [0.1, 0.9]}

In [ ]:
from hops import experiment
# Run differential evolution with default parameters
experiment.evolutionary_search(spark, pytorch_mnist, args)